In [1]:
%load_ext autotime

time: 0 ns (started: 2021-06-16 18:31:08 +05:30)


In [2]:
import os
# change working directory

print('Before chdir:', os.getcwd())

os.chdir('C:\\Users\\ravigv\\Documents\\AS') 
         
print('After chdir:', os.getcwd())

Before chdir: C:\Users\ravigv
After chdir: C:\Users\ravigv\Documents\AS
time: 0 ns (started: 2021-06-16 18:31:11 +05:30)


In [3]:
import pandas as pd
import numpy as np
import re

from scipy.sparse import csr_matrix
from typing import List

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sparse_dot_topn import awesome_cossim_topn
import yake

import warnings
warnings.filterwarnings("ignore")

time: 42.1 s (started: 2021-06-16 18:31:14 +05:30)


In [4]:
def cosine_similarity(from_vector: np.ndarray,
                      to_vector: np.ndarray,
                      from_list: List[str],
                      from_list_id: List[str],
                      to_list: List[str],
                      to_key_list: List[str],
                      nbest,
                      min_similarity: float = 0) -> pd.DataFrame:
    
    if nbest != None:
        if int(nbest) >  len(to_list):
            raise ValueError('best choice must be less than to_list')
    else:
        nbest = int(1)

    if isinstance(to_vector, np.ndarray):
        to_vector = csr_matrix(to_vector)
    if isinstance(from_vector, np.ndarray):
        from_vector = csr_matrix(from_vector)

    # There is a bug with awesome_cossim_topn that when to_vector and from_vector
    # have the same shape, setting topn to 1 does not work. Apparently, you need
    # to it at least to 2 for it to work

    if int(nbest) <= 1:
        similarity_matrix = awesome_cossim_topn(from_vector, to_vector.T, 2, min_similarity)
    elif int(nbest) > 1:
        similarity_matrix = awesome_cossim_topn(from_vector, to_vector.T, nbest, min_similarity)

    if from_list == to_list:
        similarity_matrix = similarity_matrix.tolil()
        similarity_matrix.setdiag(0.)
        similarity_matrix = similarity_matrix.tocsr()

    if int(nbest) <= 1:
        indices = np.array(similarity_matrix.argmax(axis=1).T).flatten()
        similarity = similarity_matrix.max(axis=1).toarray().T.flatten()
    elif int(nbest) > 1:
        similarity = np.flip(np.take_along_axis(similarity_matrix.toarray(), np.argsort(similarity_matrix.toarray(), axis =1), axis=1) [:,-int(nbest):], axis = 1)
        indices = np.flip(np.argsort(np.array(similarity_matrix.toarray()), axis =1)[:,-int(nbest):], axis = 1)
            
    
    if int(nbest) <= 1:
        matches = [to_list[idx] for idx in indices.flatten()]
        key_matches = [to_key_list[idx] for idx in indices.flatten()]
        matches = pd.DataFrame(np.vstack((from_list,from_list_id, matches, key_matches, similarity)).T, columns=["From", 'id', "To", "Key", "Similarity"])
        matches.Similarity = matches.Similarity.astype(float)
        matches.loc[matches.Similarity < 0.001, "To"] = None
        matches.loc[matches.Similarity < 0.001, "Key"] = None
    else:
        matches = [np.array([to_list[idx] for idx in l]) for l in indices] ##In progress
        key_matches = [np.array([to_key_list[idx] for idx in l]) for l in indices] ##In progress
        column = []
        column.append("To")
        for i in range(int(nbest) - 1):
            column.append("BestMatch" + "__" + str(i+1))
        column.append("Key")
        for j in range(int(nbest) - 1):
            column.append("Key" + "__" + str(j+1))
        column.append("Similarity")
        for j in range(int(nbest) - 1):
            column.append("Similarity" + "__" + str(j+1))
            
        matches = pd.concat([pd.DataFrame({'From' : from_list}),pd.DataFrame({'id' : from_list_id}), pd.DataFrame(np.hstack((matches, key_matches, similarity)), columns= column)], axis =1)
        matches.Similarity = matches.Similarity.astype(float)
        matches.loc[matches.Similarity < 0.001, "To"] = None
        matches.loc[matches.Similarity < 0.001, "Key"] = None
        for i in range(int(nbest) - 1):
            matches.loc[matches.Similarity < 0.001, "BestMatch" + "__" + str(i+1)] = None
            matches.loc[matches.Similarity < 0.001, "Key" + "__" + str(i+1)] = None
        
    return matches






time: 0 ns (started: 2021-06-16 18:32:50 +05:30)


In [5]:
# Keyword or ngram extraction
def kw_extractor(text, language = "en", max_ngram_size = 3,  deduplication_threshold = 0.9, numOfKeywords = 10, features = None):
    text = text
    custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=features)
    keywords = custom_kw_extractor.extract_keywords(text)
#     return keywords
    return ' '.join(list(set(" ".join([x[0] for x in keywords]).split())))


def _clean_string(string: str) -> str:
    """ Only keep alphanumerical characters """
    string = re.sub(r'[^A-Za-z0-9 ]+', '', string.lower())
    string = re.sub('\s+', ' ', string).strip()
    return string

def _create_ngrams(string: str) -> List[str]:
    n_gram_range=(3, 3)
    string = _clean_string(string)
    result = []
    for n in range(n_gram_range[0], n_gram_range[1]+1):
        ngrams = zip(*[string[i:] for i in range(n)])
        ngrams = [''.join(ngram) for ngram in ngrams if ' ' not in ngram]
        result.extend(ngrams)
    return result



def getRelevantIntent(pInputData, pQueryData, pKeyCol, pDescCol, pTopicCol, nbest = 1):
    from_list = pInputData[pTopicCol].tolist()
    from_list_id = pInputData["ticket_no"].tolist()
    to_list = pQueryData[pDescCol].values.astype('U').tolist()
    Train_CC = pQueryData[pKeyCol].values.astype('U').tolist()
    nbest = int(nbest)
    vectorizer = TfidfVectorizer(min_df=1, analyzer=_create_ngrams).fit(to_list + from_list)
    X = vectorizer.transform(to_list)
    Y = vectorizer.transform(from_list)
    to_vector = csr_matrix(X)
    from_vector = csr_matrix(Y)
    matches = cosine_similarity(Y, X, from_list,from_list_id, to_list, Train_CC, nbest)
    return matches

time: 0 ns (started: 2021-06-16 18:33:53 +05:30)


In [41]:
qdf = pd.read_excel('finaloutput.xlsx')

qdf1 = qdf.drop(['level3', 'status'], axis = 1)
qdf1 = qdf1.dropna(subset = ['Sample'])
qdf1.head()

,trainingid,account,ticketno,description,level1,level2,Sample
0,260,Allegis,INC0898710,connected - report or dashboard data looks out...,Connected ATS issue,Report or Dashboard Data Looks Out of Date or ...,connected dashboard data leader dashboard acco...
1,260,Allegis,INC0908771,connected ats - duplicate candidate merge issu...,Connected ATS issue,Unable to merge candidates,connected duplicate candidate merge candidate ...
2,260,Allegis,TASK1176571,"peoplesoft - remove access request,user tiffan...",Access request,PS_SECURITY,access tiffany access
3,260,Allegis,INC0910365,"connected drz cards missing on board,vms req t...",Missing,Connected DRZ Cards on board,connected missing board
4,260,Allegis,INC0909029,earnings code tse for position 1 with eff date...,Error,Component,earnings code position longer earnings code po...


time: 58.1 s (started: 2021-06-16 22:08:17 +05:30)


In [42]:
# qdf["ticketno"].fillna("MIS001", inplace = True) 
# qdf["description"].fillna("zzzzzz", inplace = True) 
# qdf["level1"].fillna("other", inplace = True)
# qdf["level2"].fillna("other", inplace = True)


def fillmissing(df, colst, fillst):
    df[colst[0]].fillna(fillst[0], inplace = True) 
    df[colst[1]].fillna(fillst[1], inplace = True) 
    df[colst[2]].fillna(fillst[2], inplace = True)
    df[colst[3]].fillna(fillst[3], inplace = True)
    return df
    
    
    
    

fillmissing(qdf, ['ticketno', 'description', 'level1', 'level2'], ['MIS001', 'zzzzzz', 'other', 'other'])
    

,trainingid,account,ticketno,description,level1,level2,level3,status,Sample
0,260,Allegis,INC0898710,connected - report or dashboard data looks out...,Connected ATS issue,Report or Dashboard Data Looks Out of Date or ...,NaN,NaN,connected dashboard data leader dashboard acco...
1,260,Allegis,INC0908771,connected ats - duplicate candidate merge issu...,Connected ATS issue,Unable to merge candidates,NaN,NaN,connected duplicate candidate merge candidate ...
2,260,Allegis,TASK1176571,"peoplesoft - remove access request,user tiffan...",Access request,PS_SECURITY,NaN,NaN,access tiffany access
3,260,Allegis,INC0910365,"connected drz cards missing on board,vms req t...",Missing,Connected DRZ Cards on board,NaN,NaN,connected missing board
4,260,Allegis,INC0909029,earnings code tse for position 1 with eff date...,Error,Component,NaN,NaN,earnings code position longer earnings code po...
...,...,...,...,...,...,...,...,...,...
693837,260,Allegis,INC0895160,connected - report or dashboard data looks out...,Connected ATS issue,Report or Dashboard Data Looks Out of Date or ...,NaN,NaN,connected dashboard data showing connected
693838,260,Allegis,TASK1169341,"rws - update esf status,user said that she acc...",Update request,ESF Status,NaN,NaN,update accidentally backed contract
693839,260,Allegis,INC0904362,"connected ats - how to questions,user said whe...",Connected ATS questions,Others,NaN,NaN,connected view candidate user result
693840,260,Allegis,INC0912818,"connected ats - how to questions,user save pro...",Connected ATS questions,Others,NaN,NaN,connected save submittal showing


time: 187 ms (started: 2021-06-16 22:36:16 +05:30)


In [43]:
# def gby(df, levels):
#     df.groupby([levels])['Sample'].agg(';'.join).reset_index(name='comb_desc')



# qdf1['lvl12'] = qdf1[['level1', 'level2']].agg('__'.join, axis=1).astype('category')
# qdf1
print(qdf.isnull().sum(axis = 0))

print('###############################')

print(qdf.isnull().sum(axis = 0))



print('###############################')
print(qdf.info())

trainingid          0
account             0
ticketno            0
description         0
level1              0
level2              0
level3         693842
status         693842
Sample          49721
dtype: int64
###############################
trainingid          0
account             0
ticketno            0
description         0
level1              0
level2              0
level3         693842
status         693842
Sample          49721
dtype: int64
###############################
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 693842 entries, 0 to 693841
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   trainingid   693842 non-null  int64  
 1   account      693842 non-null  object 
 2   ticketno     693842 non-null  object 
 3   description  693842 non-null  object 
 4   level1       693842 non-null  object 
 5   level2       693842 non-null  object 
 6   level3       0 non-null       float64
 7   status      

In [40]:
int_cols = qdf.select_dtypes(include=['int64']).columns
str_cols = qdf.select_dtypes(include=['object']).columns

qdf.loc[:, int_cols] = qdf.loc[:, int_cols].fillna('MIS001')
qdf.loc[:, str_cols] = qdf.loc[:, str_cols].fillna('other')

qdf

,trainingid,account,ticketno,description,level1,level2,level3,status,Sample
0,260,Allegis,INC0898710,connected - report or dashboard data looks out...,Connected ATS issue,Report or Dashboard Data Looks Out of Date or ...,other,NaN,connected dashboard data leader dashboard acco...
1,260,Allegis,INC0908771,connected ats - duplicate candidate merge issu...,Connected ATS issue,Unable to merge candidates,other,NaN,connected duplicate candidate merge candidate ...
2,260,Allegis,TASK1176571,"peoplesoft - remove access request,user tiffan...",Access request,PS_SECURITY,other,NaN,access tiffany access
3,260,Allegis,INC0910365,"connected drz cards missing on board,vms req t...",Missing,Connected DRZ Cards on board,other,NaN,connected missing board
4,260,Allegis,INC0909029,earnings code tse for position 1 with eff date...,Error,Component,other,NaN,earnings code position longer earnings code po...
...,...,...,...,...,...,...,...,...,...
693837,260,Allegis,INC0895160,connected - report or dashboard data looks out...,Connected ATS issue,Report or Dashboard Data Looks Out of Date or ...,other,NaN,connected dashboard data showing connected
693838,260,Allegis,TASK1169341,"rws - update esf status,user said that she acc...",Update request,ESF Status,other,NaN,update accidentally backed contract
693839,260,Allegis,INC0904362,"connected ats - how to questions,user said whe...",Connected ATS questions,Others,other,NaN,connected view candidate user result
693840,260,Allegis,INC0912818,"connected ats - how to questions,user save pro...",Connected ATS questions,Others,other,NaN,connected save submittal showing


time: 547 ms (started: 2021-06-16 21:39:29 +05:30)


In [32]:
qdf['level3'].fillna('other', inplace=True)

qdf

,trainingid,account,ticketno,description,level1,level2,level3,status,Sample
0,260,Allegis,INC0898710,connected - report or dashboard data looks out...,Connected ATS issue,Report or Dashboard Data Looks Out of Date or ...,other,NaN,connected dashboard data leader dashboard acco...
1,260,Allegis,INC0908771,connected ats - duplicate candidate merge issu...,Connected ATS issue,Unable to merge candidates,other,NaN,connected duplicate candidate merge candidate ...
2,260,Allegis,TASK1176571,"peoplesoft - remove access request,user tiffan...",Access request,PS_SECURITY,other,NaN,access tiffany access
3,260,Allegis,INC0910365,"connected drz cards missing on board,vms req t...",Missing,Connected DRZ Cards on board,other,NaN,connected missing board
4,260,Allegis,INC0909029,earnings code tse for position 1 with eff date...,Error,Component,other,NaN,earnings code position longer earnings code po...
...,...,...,...,...,...,...,...,...,...
693837,260,Allegis,INC0895160,connected - report or dashboard data looks out...,Connected ATS issue,Report or Dashboard Data Looks Out of Date or ...,other,NaN,connected dashboard data showing connected
693838,260,Allegis,TASK1169341,"rws - update esf status,user said that she acc...",Update request,ESF Status,other,NaN,update accidentally backed contract
693839,260,Allegis,INC0904362,"connected ats - how to questions,user said whe...",Connected ATS questions,Others,other,NaN,connected view candidate user result
693840,260,Allegis,INC0912818,"connected ats - how to questions,user save pro...",Connected ATS questions,Others,other,NaN,connected save submittal showing


time: 204 ms (started: 2021-06-16 20:04:44 +05:30)


In [14]:
qdf1.groupby(['level1','level2'])['Sample'].agg(';'.join).reset_index(name='comb_desc')

,level1,level2,comb_desc
0,2020 Annual Article Revalidation,Change Owner,annual article revalidation article;annual art...
1,2020 Annual Article Revalidation,KB article,annual article revalidation meta;annual articl...
2,2020 Annual Article Revalidation,Update Article,annual article revalidation;annual article rev...
3,2FA Token enrollment request,Existing user_new device,user enroll token device;user enroll token dev...
4,2FA Token enrollment request,First time user,time token enrollment;time token enrollment;ti...
...,...,...,...
15777,webMethods subprocess Error,Oracle Concurrent,error raise description;error raise assign;err...
15778,webMethods subprocess Error,TuplesBlockInsertOrUpdateTB,error integration interface transfer applicati...
15779,wrong restaurant,EID,showing wrong restaurant;showing wrong restaurant
15780,wrong restaurant,MyIDM,wrong restaurant;wrong restaurant;incident rai...


time: 531 ms (started: 2021-06-16 18:56:59 +05:30)


In [12]:
# gby(qdf1, ['level1', 'level2', 'level3'])

time: 0 ns (started: 2021-06-16 18:56:11 +05:30)


In [8]:
qdf2 = qdf1.groupby('level1')['Sample'].agg(';'.join).reset_index(name='comb_desc')
# tdf2['count'] = tdf2['comb_desc'].apply(lambda x: len(x.split(';')))

qdf2

,level1,comb_desc
0,2020 Annual Article Revalidation,annual article revalidation annual article rev...
1,2FA Token enrollment request,user enroll token device;time token enrollment...
2,365 Day Aging,aging;aging;aging;aging;aging;aging;aging;agin...
3,3POD issue,void order;user pod menu;user increase;user;un...
4,997 Overdue Issue,overdue;overdue;overdue;overdue;overdue;overdu...
...,...,...
3670,wflw_STG_INV_ITEM_DETAILS failed,multiple assigned category setup
3671,wrong input,wrong input;assist wrong input;assist wrong input
3672,wrong qty,wrong;wrong;wrong;aging sending wrong mail;wro...
3673,wrong restaurant,wrong restaurant;wrong restaurant;showing wron...


time: 282 ms (started: 2021-06-15 14:19:04 +05:30)


In [9]:
qdf2["topkw"]  = qdf2["comb_desc"].apply(lambda x: kw_extractor(x, deduplication_threshold = 0.3, numOfKeywords = 25))
qdf2

,level1,comb_desc,topkw
0,2020 Annual Article Revalidation,annual article revalidation annual article rev...,file retired feel confirmed corrected master c...
1,2FA Token enrollment request,user enroll token device;time token enrollment...,enroll user maria account request phone site s...
2,365 Day Aging,aging;aging;aging;aging;aging;aging;aging;agin...,aging
3,3POD issue,void order;user pod menu;user increase;user;un...,menu unavailable order user topping sandwich c...
4,997 Overdue Issue,overdue;overdue;overdue;overdue;overdue;overdu...,overdue
...,...,...,...
3670,wflw_STG_INV_ITEM_DETAILS failed,multiple assigned category setup,multiple assigned setup category
3671,wrong input,wrong input;assist wrong input;assist wrong input,wrong assist input
3672,wrong qty,wrong;wrong;wrong;aging sending wrong mail;wro...,mail aging wrong sending
3673,wrong restaurant,wrong restaurant;wrong restaurant;showing wron...,wrong locator incident search result raised sh...


time: 15min 58s (started: 2021-06-15 14:19:04 +05:30)


In [29]:
tdf = pd.read_excel('topicsub.xlsx')

tdf['ticket_no'] = tdf['ticket_no'].astype('str')
tdf

,ticket_no,ticket_type,ticket_description,preprocessed_description,solvedworkgroup,topic,uploadid,topic_confidence_level,action_flag,action_status,split_number
0,21983985,Service Request,mPOS: PR 14314 - Transactions not posting / Mi...,post miss,SAP BW AS,post__remorse__reprint__variance,6,0.0673,A,Initial,0
1,21983959,Incident,Quikview: Transaction did not post to customer...,transaction post account,SAP BW AS,retail__store__account__transaction,6,0.0702,A,Initial,0
2,21982340,Incident,Retail Inventory discrepancy / Issue (6%),retail inventory discrepancy,SAP BW AS,discrepancy__signature__rebellion__customer,6,0.0660,A,Initial,0
3,21991703,Incident,Retail Inventory discrepency / Issue (6%),retail inventory,SAP BW AS,inventory__pier__rebellion__customer,6,0.0545,A,Initial,0
4,21992515,Incident,RE: SAP 2940 - Chicago Signature Store - Missi...,signature store miss transaction pier,SAP BW AS,retail__store__account__transaction,6,0.0717,A,Initial,0
...,...,...,...,...,...,...,...,...,...,...,...
3489,27717169,Incident,Coverage Device Pending Fulfillment,coverage device pende fulfillment,SAP O2C,pende__coverage__device__fulfillment,1,0.0586,A,Initial,0
3490,22375609,Incident,Retail Inventory discrepency / Issue (6%),retail inventory,SAP BW AS,inventory__retail__showing__match,10,0.0865,A,Initial,0
3491,27717569,Incident,Coverage Device Pending Fulfillment,coverage device pende fulfillment,SAP O2C,fulfillment__pende__device__validation,1,0.0648,A,Initial,0
3492,27725806,Incident,EIP stuck in Pending-Shipped,stuck,SAP O2C,error__stuck__clear__block,1,0.0686,A,Initial,0


time: 421 ms (started: 2021-06-15 14:57:13 +05:30)


In [51]:
tdf1 = tdf
tdf1 =tdf1.drop_duplicates(subset = ['ticket_no'])
tdf1 = tdf1.reset_index(drop = True)
tdf1

,ticket_no,ticket_type,ticket_description,preprocessed_description,solvedworkgroup,topic,uploadid,topic_confidence_level,action_flag,action_status,split_number
0,21983985,Service Request,mPOS: PR 14314 - Transactions not posting / Mi...,post miss,SAP BW AS,post__remorse__reprint__variance,6,0.0673,A,Initial,0
1,21983959,Incident,Quikview: Transaction did not post to customer...,transaction post account,SAP BW AS,retail__store__account__transaction,6,0.0702,A,Initial,0
2,21982340,Incident,Retail Inventory discrepancy / Issue (6%),retail inventory discrepancy,SAP BW AS,discrepancy__signature__rebellion__customer,6,0.0660,A,Initial,0
3,21991703,Incident,Retail Inventory discrepency / Issue (6%),retail inventory,SAP BW AS,inventory__pier__rebellion__customer,6,0.0545,A,Initial,0
4,21992515,Incident,RE: SAP 2940 - Chicago Signature Store - Missi...,signature store miss transaction pier,SAP BW AS,retail__store__account__transaction,6,0.0717,A,Initial,0
...,...,...,...,...,...,...,...,...,...,...,...
2877,27717169,Incident,Coverage Device Pending Fulfillment,coverage device pende fulfillment,SAP O2C,pende__coverage__device__fulfillment,1,0.0586,A,Initial,0
2878,22375609,Incident,Retail Inventory discrepency / Issue (6%),retail inventory,SAP BW AS,inventory__retail__showing__match,10,0.0865,A,Initial,0
2879,27717569,Incident,Coverage Device Pending Fulfillment,coverage device pende fulfillment,SAP O2C,fulfillment__pende__device__validation,1,0.0648,A,Initial,0
2880,27725806,Incident,EIP stuck in Pending-Shipped,stuck,SAP O2C,error__stuck__clear__block,1,0.0686,A,Initial,0


time: 16 ms (started: 2021-06-15 17:11:07 +05:30)


In [20]:
# input data == Topic df
# query data == intent df
# getRelevantIntent(pInputData, pQueryData, pKeyCol, pDescCol, pTopicCol, nbest = 1)

idf = getRelevantIntent(tdf, qdf2, 'level1','topkw', 'topic', nbest = 1)
idf



# matches = cosine_similarity(Y, X, test,test_id, train, train_CC, 1, 0)
# matches

,From,id,To,Key,Similarity
0,post__remorse__reprint__variance,21983985,inventory variance hand,Inventory variance issue,0.342085
1,retail__store__account__transaction,21983959,reactivate account,Account Reactivation Request,0.391066
2,discrepancy__signature__rebellion__customer,21982340,customer update,Customer Update Request,0.268482
3,inventory__pier__rebellion__customer,21991703,customer update,Customer Update Request,0.287502
4,retail__store__account__transaction,21992515,reactivate account,Account Reactivation Request,0.391066
...,...,...,...,...,...
3489,pende__coverage__device__fulfillment,27717169,fulfillment,Under Fulfillment Request,0.494122
3490,inventory__retail__showing__match,22375609,showing,Replenishment issue,0.431071
3491,fulfillment__pende__device__validation,27717569,validation,OMR Validation Request,0.529843
3492,error__stuck__clear__block,27725806,stuck,Stuck TRFC issue,0.394161


time: 813 ms (started: 2021-06-15 14:46:50 +05:30)


In [52]:
idf1 = idf
idf1 =idf1.drop_duplicates(subset = ['id']).reset_index(drop = True)
len(idf1)

2882

time: 0 ns (started: 2021-06-15 17:11:40 +05:30)


In [45]:
# final = pd.merge(tdf, idf, how = 'inner', on = ['ticket_no', 'id'])
# final

print(len(tdf))
print(len(idf))


print(tdf)
tdf.to_excel('tdf.xlsx')

3494
3494
     ticket_no      ticket_type  \
0     21983985  Service Request   
1     21983959         Incident   
2     21982340         Incident   
3     21991703         Incident   
4     21992515         Incident   
...        ...              ...   
3489  27717169         Incident   
3490  22375609         Incident   
3491  27717569         Incident   
3492  27725806         Incident   
3493  27728131  Service Request   

                                     ticket_description  \
0     mPOS: PR 14314 - Transactions not posting / Mi...   
1     Quikview: Transaction did not post to customer...   
2             Retail Inventory discrepancy / Issue (6%)   
3             Retail Inventory discrepency / Issue (6%)   
4     RE: SAP 2940 - Chicago Signature Store - Missi...   
...                                                 ...   
3489                Coverage Device Pending Fulfillment   
3490          Retail Inventory discrepency / Issue (6%)   
3491                Coverage Device Pe

In [44]:
print(idf)
idf.to_excel('idf.xlsx')

                                             From        id  \
0                post__remorse__reprint__variance  21983985   
1             retail__store__account__transaction  21983959   
2     discrepancy__signature__rebellion__customer  21982340   
3            inventory__pier__rebellion__customer  21991703   
4             retail__store__account__transaction  21992515   
...                                           ...       ...   
3489         pende__coverage__device__fulfillment  27717169   
3490            inventory__retail__showing__match  22375609   
3491       fulfillment__pende__device__validation  27717569   
3492                   error__stuck__clear__block  27725806   
3493                      run__aal__order__status  27728131   

                           To                                Key  Similarity  
0     inventory variance hand           Inventory variance issue    0.342085  
1          reactivate account       Account Reactivation Request    0.391066  
2     

In [55]:
dfm = pd.merge(idf1, tdf,  how = 'left', left_on='id', right_on='ticket_no')
# dfm.to_excel('merge1.xlsx')

time: 47 ms (started: 2021-06-15 18:23:29 +05:30)


In [56]:
dfm

,From,id,To,Key,Similarity,ticket_no,ticket_type,ticket_description,preprocessed_description,solvedworkgroup,topic,uploadid,topic_confidence_level,action_flag,action_status,split_number
0,post__remorse__reprint__variance,21983985,inventory variance hand,Inventory variance issue,0.342085,21983985,Service Request,mPOS: PR 14314 - Transactions not posting / Mi...,post miss,SAP BW AS,post__remorse__reprint__variance,6,0.0673,A,Initial,0
1,post__remorse__reprint__variance,21983985,inventory variance hand,Inventory variance issue,0.342085,21983985,Service Request,mPOS: PR 14314 - Transactions not posting / Mi...,post miss,SAP BW AS,post__complete__posting__customer,7,0.0652,A,Initial,0
2,post__remorse__reprint__variance,21983985,inventory variance hand,Inventory variance issue,0.342085,21983985,Service Request,mPOS: PR 14314 - Transactions not posting / Mi...,post miss,SAP BW AS,post__complete__posting__customer,2,0.0652,A,Initial,0
3,post__remorse__reprint__variance,21983985,inventory variance hand,Inventory variance issue,0.342085,21983985,Service Request,mPOS: PR 14314 - Transactions not posting / Mi...,post miss,SAP BW AS,post__unable__account__store,3,0.0673,A,Initial,0
4,post__remorse__reprint__variance,21983985,inventory variance hand,Inventory variance issue,0.342085,21983985,Service Request,mPOS: PR 14314 - Transactions not posting / Mi...,post miss,SAP BW AS,post__count__store__variance,4,0.0673,A,Initial,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3489,pende__coverage__device__fulfillment,27717169,fulfillment,Under Fulfillment Request,0.494122,27717169,Incident,Coverage Device Pending Fulfillment,coverage device pende fulfillment,SAP O2C,pende__coverage__device__fulfillment,1,0.0586,A,Initial,0
3490,inventory__retail__showing__match,22375609,showing,Replenishment issue,0.431071,22375609,Incident,Retail Inventory discrepency / Issue (6%),retail inventory,SAP BW AS,inventory__retail__showing__match,10,0.0865,A,Initial,0
3491,fulfillment__pende__device__validation,27717569,validation,OMR Validation Request,0.529843,27717569,Incident,Coverage Device Pending Fulfillment,coverage device pende fulfillment,SAP O2C,fulfillment__pende__device__validation,1,0.0648,A,Initial,0
3492,error__stuck__clear__block,27725806,stuck,Stuck TRFC issue,0.394161,27725806,Incident,EIP stuck in Pending-Shipped,stuck,SAP O2C,error__stuck__clear__block,1,0.0686,A,Initial,0


time: 31 ms (started: 2021-06-15 18:23:34 +05:30)


In [16]:
type(idf['id'][0])

str

time: 15 ms (started: 2021-06-15 14:41:21 +05:30)


In [17]:
type(tdf['ticket_no'][0])

int

time: 0 ns (started: 2021-06-15 14:41:49 +05:30)


In [54]:
dfm

,From,id,To,Key,Similarity,ticket_no,ticket_type,ticket_description,preprocessed_description,solvedworkgroup,topic,uploadid,topic_confidence_level,action_flag,action_status,split_number
0,post__remorse__reprint__variance,21983985,inventory variance hand,Inventory variance issue,0.342085,21983985,Service Request,mPOS: PR 14314 - Transactions not posting / Mi...,post miss,SAP BW AS,post__remorse__reprint__variance,6,0.0673,A,Initial,0
1,retail__store__account__transaction,21983959,reactivate account,Account Reactivation Request,0.391066,21983959,Incident,Quikview: Transaction did not post to customer...,transaction post account,SAP BW AS,retail__store__account__transaction,6,0.0702,A,Initial,0
2,discrepancy__signature__rebellion__customer,21982340,customer update,Customer Update Request,0.268482,21982340,Incident,Retail Inventory discrepancy / Issue (6%),retail inventory discrepancy,SAP BW AS,discrepancy__signature__rebellion__customer,6,0.0660,A,Initial,0
3,inventory__pier__rebellion__customer,21991703,customer update,Customer Update Request,0.287502,21991703,Incident,Retail Inventory discrepency / Issue (6%),retail inventory,SAP BW AS,inventory__pier__rebellion__customer,6,0.0545,A,Initial,0
4,retail__store__account__transaction,21992515,reactivate account,Account Reactivation Request,0.391066,21992515,Incident,RE: SAP 2940 - Chicago Signature Store - Missi...,signature store miss transaction pier,SAP BW AS,retail__store__account__transaction,6,0.0717,A,Initial,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2877,pende__coverage__device__fulfillment,27717169,fulfillment,Under Fulfillment Request,0.494122,27717169,Incident,Coverage Device Pending Fulfillment,coverage device pende fulfillment,SAP O2C,pende__coverage__device__fulfillment,1,0.0586,A,Initial,0
2878,inventory__retail__showing__match,22375609,showing,Replenishment issue,0.431071,22375609,Incident,Retail Inventory discrepency / Issue (6%),retail inventory,SAP BW AS,inventory__retail__showing__match,10,0.0865,A,Initial,0
2879,fulfillment__pende__device__validation,27717569,validation,OMR Validation Request,0.529843,27717569,Incident,Coverage Device Pending Fulfillment,coverage device pende fulfillment,SAP O2C,fulfillment__pende__device__validation,1,0.0648,A,Initial,0
2880,error__stuck__clear__block,27725806,stuck,Stuck TRFC issue,0.394161,27725806,Incident,EIP stuck in Pending-Shipped,stuck,SAP O2C,error__stuck__clear__block,1,0.0686,A,Initial,0


time: 31 ms (started: 2021-06-15 17:12:00 +05:30)
